# Модель "Цены и себестоимость"

In [1]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats.mstats import gmean 
from datetime import datetime as dt

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

from source_data.src import db_source, RowTypes
import source_data.prepare as prep

iFirstFactYear=2008
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

In [12]:
class prices_cost:
    lstFields=['Invest_Iq', 'Retail_Iq', 'ZPR_I', 'CPIAv', 'squarekp_ipot_x', 'price1mall_alt_x', 'price1mvtor_alt_x', 
               'squarekp_ss_x', 'squareddy_x', 'ProjectsPrivate_x', 'price1mperv_alt_x', 'seb1m_x', 'price1mperv_alt_x', 
              'p_LabProdOnCost', 'p_GovPolOnCost', 'p_LabProdOnCost', 'p_GovPolOnCost', 'price1mperv_alt_x', 'price1mddy_alt_x']
#     lstExogPFields=['p_LabProdOnCost', 'p_GovPolOnCost']
#     lstSVODFields=['p_LabProdOnCost', 'p_GovPolOnCost', 'price1mperv_alt_x']

    
# блок внутримодельных констант
    cElastiCo     = 0.05 # Эластичность коррекции 
    cErrLevelDown = 0.08 # Допустимая ошибка вниз
    cErrLevelUp   = 0.01 # Допустимая ошибка вверх
    cPriceDown    = 0.15 # Наиболее сильное допустимое снижение цены ДДУ за год
    cPriceUp      = 0.15 # Наиболее сильный допустимый рост цены ДДУ за год
    cLtASoSHUC    = 0.75  # Долгосрочный средний уровень доли реализованного строящегося жилья

    def __init__(self):
        assert False, 'you can\'t create variables of rpices_cost class!'
    
    def MakeFrame():
        srcAct   = db_source(common.strYearDBPath, RowTypes.FACT, prices_cost.lstFields)
        srcExogR = db_source(common.strExYearDBPath, RowTypes.EXOG_R, prices_cost.lstFields)
        srcExogP = db_source(common.strExParamDBPath, RowTypes.EXOG_P, prices_cost.lstFields)
        srcSvod  = db_source(common.strSvodDBPath, RowTypes.MODEL, prices_cost.lstFields)
        
        prices_cost.pdfWork=common.combine_frames(srcAct, srcExogR, srcExogP, srcSvod)
        
        return prices_cost.pdfWork


In [13]:
prices_cost.MakeFrame()

,CPIAv,Invest_Iq,ProjectsPrivate_x,Retail_Iq,ZPR_I,p_GovPolOnCost,p_LabProdOnCost,price1mall_alt_x,price1mddy_alt_x,price1mperv_alt_x,price1mvtor_alt_x,seb1m_x,squareddy_x,squarekp_ipot_x,squarekp_ss_x
date,,,,,,,,,,,,,,,
2004,1.108812,1.168000,27.254825,1.133000,1.106000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,1.127055,1.102000,30.726575,1.128000,1.126000,NaN,NaN,21.048567,18.491693,20.777183,21.319952,13.812000,NaN,NaN,NaN
2006,1.096830,1.178000,31.297480,1.141000,1.133000,NaN,NaN,28.597165,24.937897,28.020109,29.174222,16.840000,NaN,NaN,NaN
2007,1.089881,1.238000,29.248635,1.161000,1.172000,NaN,NaN,37.938493,32.765903,36.815621,39.061366,20.720000,NaN,NaN,NaN
2008,1.141048,1.095000,29.003730,1.137000,1.115000,NaN,NaN,44.576706,37.788916,42.459456,46.693956,26.622000,NaN,14.763884,65.564414
2009,1.116977,0.865000,31.282002,0.949000,0.965000,NaN,NaN,42.565501,36.473861,40.981867,44.149136,30.312000,NaN,5.924064,60.182264
2010,1.068508,1.063000,34.126605,1.065000,1.052000,NaN,NaN,41.870585,35.853806,40.285175,43.455994,31.877000,8.765628,13.778887,84.407359
2011,1.084634,1.108000,38.266952,1.071000,1.028000,NaN,NaN,44.815618,37.897473,42.581430,47.049805,33.320000,13.589131,22.686387,101.337510
2012,1.050667,1.068000,43.570800,1.063000,1.084000,NaN,NaN,50.629951,41.865338,47.039706,54.220195,34.354000,18.701917,28.127839,106.175779


## Среднегодовая цена кв.м по ДДУ: price1mddy\_alt\_x

### Расчет переменных

#### 1. Допустимый рост цены ДДУ за год:   \_m1\_price\_limit_y 

Для интервала дат $ iFirstFactYear \leq y \leq iLastFactYear$: 

$ \_m1\_price\_limit_y = price1mddy\_alt\_x_y $


Для интервала дат $ iLastFactYear < y \leq iLastForecastYear$: 

$$ \begin{equation*}
\_m1\_price\_limit_y =  
 \begin{cases}
   \_price\_min_y; \quad \text{при} \quad \ price1mddy\_alt\_x_y \leq \_price\_min_y \\
   \_price\_max_y; \quad \text{при} \quad \ price1mddy\_alt\_x_y \geq \_price\_max_y \\
   price1mddy\_alt\_x_y; \quad \text{при} \_price\_min_y \leq price1mddy\_alt\_x_y \leq \_price\_max_y \\
 \end{cases}
\end{equation*} $$

где:

   $ \_m1\_price\_min_y = (1-cPriceDown)*\_m1\_price\_limit_{y-1} $   
   $ \_m1\_price\_max_y = (1+cPriceUp)*\_m1\_price\_limit_{y-1} $   



In [20]:
def _lower_price(x):
    return (1-prices_cost.cPriceDown)*x

def _upper_price(x):
    return (1+prices_cost.cPriceUp)*x

prices_cost.pdfWork['_m1_price_limit'] = (prices_cost.pdfWork['price1mddy_alt_x']
                                          .clip(lower=_lower_price(prices_cost.pdfWork['_m1_price_limit'].shift(1)), 
                                                upper=_upper_price(prices_cost.pdfWork['_m1_price_limit'].shift(1))))

prices_cost.pdfWork.loc[iFirstFactYear:iLastFactYear, '_m1_price_limit']=prices_cost.pdfWork.loc[iFirstFactYear:iLastFactYear, 'price1mddy_alt_x']

prices_cost.pdfWork[['_m1_price_limit', 'price1mddy_alt_x']]

,_m1_price_limit,price1mddy_alt_x
date,,
2004,NaN,NaN
2005,NaN,18.491693
2006,NaN,24.937897
2007,NaN,32.765903
2008,37.788916,37.788916
2009,36.473861,36.473861
2010,35.853806,35.853806
2011,37.897473,37.897473
2012,41.865338,41.865338
